In [1]:
from KaBOBInterface import KaBOBInterface
import networkx as nx
from franz.openrdf.vocabulary import RDFS, OWL, RDF
from franz.openrdf.query.query import QueryLanguage
import matplotlib.pyplot as plt
credentials_file = "KaBOB_credentials.txt"

%matplotlib inline

In [2]:
def decompose_statements(statements):
    for statement in statements:
        print("S: %s P: %s O: %s" % (interface.get_label(statement.getSubject()), 
                                     interface.get_label(statement.getPredicate()), 
                                     interface.get_label(statement.getObject())))

In [3]:
interface = KaBOBInterface(credentials_file)
interface.__enter__()

DEBUG:KaBOBInterface:Connecting to repository --host:'amc-tantor.ucdenver.pvt' port:10037


In [8]:
ldl_uri = "ice:REACTOME_R-HSA-8964038"

ldl_path = interface.get_bio_node(ldl_uri)
interface.mopify_pathway(ldl_path)

DEBUG:KaBOBInterface:> subclass of low-density lipoprotein particle clearance
DEBUG:KaBOBInterface:	> low-density lipoprotein particle clearance
DEBUG:KaBOBInterface:		> plasma lipoprotein particle clearance
DEBUG:KaBOBInterface:			> multicellular organismal process
DEBUG:KaBOBInterface:				> biological_process
DEBUG:KaBOBInterface:					> processual entity
DEBUG:KaBOBInterface:						> occurrent
DEBUG:KaBOBInterface:							> entity
DEBUG:KaBOBInterface:							< entity
DEBUG:KaBOBInterface:						< occurrent
DEBUG:KaBOBInterface:					< processual entity
DEBUG:KaBOBInterface:				< biological_process
DEBUG:KaBOBInterface:				> Saccharomyces cerevisiae
DEBUG:KaBOBInterface:					> Saccharomyces
DEBUG:KaBOBInterface:						> Saccharomycetaceae
DEBUG:KaBOBInterface:							> Saccharomycetales
DEBUG:KaBOBInterface:								> Saccharomycetes
DEBUG:KaBOBInterface:									> Saccharomycotina
DEBUG:KaBOBInterface:										> saccharomyceta
DEBUG:KaBOBInterface:											> Ascomycota
DEBUG:KaBOBInterfa

KeyboardInterrupt: 

In [9]:
decompose_statements(interface.get_statements(s=interface.get_node(ldl_uri)))

S: subclass of Reactome identifier P: denotes O: subclass of low-density lipoprotein particle clearance
S: subclass of Reactome identifier P: subClassOf O: Reactome identifier
S: subclass of Reactome identifier P: subClassOf O: ontology concept identifier


In [ ]:
def is_parent(u, v):
    return interface.mops.is_abstraction(u, v)
def label_nodes(B):
    label = None
    for u in B:
        label = {interface.mops.attribute_label: interface.mops.get_frame_label(u)}
        if "sub class of" not in label:
            return label

    return label
        
collapsed_slots = nx.quotient_graph(interface.mops.slots, is_parent, node_data=label_nodes)
print(interface.mops.slots.number_of_nodes())
print(collapsed_slots.number_of_nodes())
print(interface.mops.slots.number_of_edges())
print(collapsed_slots.number_of_edges())

In [ ]:
edge_labels = {}
for (u, v, key), label in nx.get_edge_attributes(interface.mops.slots, interface.mops.attribute_label).items():
    edge_labels[key] = str(label)

node_labels = nx.get_node_attributes(collapsed_slots, interface.mops.attribute_label)

def get_label(node):
    if isinstance(node, str):
        return node
    
    if node in node_labels:
        return str(node_labels[node])
    else: 
        return edge_labels[node]

In [ ]:
nx.write_gml(collapsed_slots, "E:/Documents/KaBOB/REACTOME/collapsed_slots.gml", get_label)